In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import sys
sys.path.insert(0, "C:/workspace/data-science//libs")
from big_numbers import contract_18_decimals_to_float

import warnings
warnings.filterwarnings('ignore')

Here will be formation of the "master" table containin info about makers on the platform and complete overview of their activity. It does not mean that the same address can not be a trader, but at the current stage here will be only data about makers

In [2]:
tokens_addresses_to_names_dict = {
    "0x86f1e0420c26a858fc203a3645dd1a36868f18e5": "vBTC",
    "0x8c835dfaa34e2ae61775e80ee29e2c724c6ae2bb": "vETH",
    "0x5faa136fc58b6136ffdaeaac320076c4865c070f": "vAVAX",
    "0xb24f50dd9918934ab2228be7a097411ca28f6c14": "vLUNA",
    "0x151bb01c79f4516c233948d69dae39869bccb737": "vSOL",
    "0x7161c3416e08abaa5cd38e68d9a28e43a694e037": "vCRV",
    "0x333b1ea429a88d0dd48ce7c06c16609cd76f43a8": "vSAND",
    "0x2f198182ec54469195a4a06262a9431a42462373": "vLINK",
    "0x5f714b5347f0b5de9f9598e39840e176ce889b9c": "vATOM",
    "0x77d0cc9568605bfff32f918c8ffaa53f72901416": "vONE",
    "0x3fb3282e3ba34a0bff94845f1800eb93cc6850d4": "vNEAR",
    "0x2db8d2db86ca3a4c7040e778244451776570359b": "vFTM",
    "0x7eada83e15acd08d22ad85a1dce92e5a257acb92": "vFLOW",
    "0xb6599bd362120dc70d48409b8a08888807050700": "vBNB",
    "0x9482aafdced6b899626f465e1fa0cf1b1418d797": "vPERP",
    "0xbe5de48197fc974600929196239e264ecb703ee8": "vMATIC",
    "0x34235c8489b06482a99bb7fcab6d7c467b92d248": "vAAVE",
    "0x9d34f1d15c22e4c0924804e2a38cbe93dfb84bc2": "vAPE"
}

# Collecting tables to form a "master" one

## Liquidity changes

In [3]:
liquidity_changes_df = pd.read_csv("liquidity_changes.csv")
liquidity_changes_df["time"] = pd.to_datetime(liquidity_changes_df["timestamp"], unit='s')
liquidity_changes_df["day_of_year"] = liquidity_changes_df["time"].dt.dayofyear
liquidity_changes_df["pool_name"] = liquidity_changes_df["base_token"].replace(tokens_addresses_to_names_dict)
liquidity_changes_df.drop(columns=["base_token", "tx_hash", "from_function_signature"], inplace=True)
liquidity_changes_df["lower_bound"] = pow(1.0001, liquidity_changes_df["lower_tick"])
liquidity_changes_df["upper_bound"] = pow(1.0001, liquidity_changes_df["upper_tick"])
liquidity_changes_df["fixed_liquidity"] = [contract_18_decimals_to_float(liquidity_record) for 
                                           liquidity_record in liquidity_changes_df["liquidity"]]
liquidity_changes_df.drop(columns=["Unnamed: 0", "quote_token", "lower_tick", "upper_tick", "liquidity", "block_number"], 
                          inplace=True)
liquidity_changes_df["central_bound"] = (liquidity_changes_df["upper_bound"] + liquidity_changes_df["lower_bound"]) / 2
liquidity_changes_df.sort_values("block_number_log_index", inplace=True)
liquidity_changes_df = liquidity_changes_df.reset_index()
liquidity_changes_df.drop(columns=["index"], inplace=True)

liquidity_changes_df

,id,maker,base,quote,quote_fee,block_number_log_index,timestamp,time,day_of_year,pool_name,lower_bound,upper_bound,fixed_liquidity,central_bound
0,0xec1e410b931f920c4aa3ae105e63463142b310fbe959...,0x9c1e0c67aa30c063f341885b12cb81cc94613fc7,0.000921,50.000000,0.000000,515552004,1638000985,2021-11-27 08:16:25,331,vBTC,2.954278e-39,3.384921e+38,0.214579,1.692461e+38
1,0x7dba9c535f6c595ab39eeec818d89b34783442bafa81...,0x9c1e0c67aa30c063f341885b12cb81cc94613fc7,0.012181,50.000000,0.000000,515554004,1638000985,2021-11-27 08:16:25,331,vETH,2.954278e-39,3.384921e+38,0.780407,1.692461e+38
2,0xc438a7cd8fee202ce2b53761811ef05f85095933b27e...,0x9c1e0c67aa30c063f341885b12cb81cc94613fc7,3.572179,200000.000000,0.000000,515555003,1638000985,2021-11-27 08:16:25,331,vBTC,5.171263e+04,5.692287e+04,35648.835497,5.431775e+04
3,0x7871b4fe6113ac31c8be911146b109575cfb9ee90b13...,0x9c1e0c67aa30c063f341885b12cb81cc94613fc7,46.254982,200000.000000,0.000000,515557003,1638000985,2021-11-27 08:16:25,331,vETH,3.895536e+03,4.313830e+03,120851.329740,4.104683e+03
4,0x653f9413a9c8df27d2800bbb71f06fd20b6d825b3d38...,0x35952141922ee99a04e016c967c3f0ad589ea958,0.100000,472.798469,0.000000,516478004,1638003992,2021-11-27 09:06:32,331,vETH,3.990155e+03,4.288026e+03,386.315245,4.139091e+03
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
130054,0xf3155f56c66c05c95611e7947304853aab760c5ff395...,0xd3fab52a0c93346f23a9b9b24fd45ea0b8dd3420,-1683.962836,0.000000,144.993911,11554393008,1655117811,2022-06-13 10:56:51,164,vAAVE,5.948926e+01,7.164952e+01,-146260.729784,6.556939e+01
130055,0xc8c4310f0bb74d949669849479190ba4b1eebee33f58...,0x377e6326a8e2ce4a856d31605ca33505ad2b57b1,1435.815543,4999.462445,0.000000,11554604003,1655117891,2022-06-13 10:58:11,164,vAPE,3.478628e-01,3.487682e+01,3917.139686,1.761234e+01
130056,0x8e7f48783a1d80dc4a058e7595f89d5d485d9d054f27...,0xd3fab52a0c93346f23a9b9b24fd45ea0b8dd3420,1708.506923,0.000000,0.000000,11554671003,1655117952,2022-06-13 10:59:12,164,vAAVE,5.877969e+01,7.037143e+01,152196.723027,6.457556e+01
130057,0xb7473ff85536e28acf2175d68acbbf68351a02dd7a7a...,0x377e6326a8e2ce4a856d31605ca33505ad2b57b1,-331.643015,-4978.735548,2.235039,11555163007,1655118198,2022-06-13 11:03:18,164,vAVAX,1.512826e+00,1.516763e+02,-1878.686957,7.659458e+01


## Open Orders

In [4]:
open_orders_df = pd.read_csv("open_orders.csv")
open_orders_df["time"] = pd.to_datetime(open_orders_df["timestamp"], unit='s')
open_orders_df["pool_name"] = open_orders_df["base_token"].replace(tokens_addresses_to_names_dict)
open_orders_df["lower_bound"] = pow(1.0001, open_orders_df["lower_tick"])
open_orders_df["upper_bound"] = pow(1.0001, open_orders_df["upper_tick"])
open_orders_df["fixed_liquidity"] = [contract_18_decimals_to_float(open_order) for 
                                           open_order in open_orders_df["liquidity"]]
open_orders_df.drop(columns=["Unnamed: 0", "base_token", "lower_tick", "upper_tick", "liquidity"], inplace=True)
open_orders_df

,id,collected_fee,collected_fee_in_lifecycle,block_number,timestamp,time,pool_name,lower_bound,upper_bound,fixed_liquidity
0,0xb3344d84ad8dd6b96301991bf12dc739e226c9cf-0x8...,0.092905,0.0,519810,1638012541,2021-11-27 11:29:01,vETH,3849.071493,4136.410359,0.000000
1,0x35952141922ee99a04e016c967c3f0ad589ea958-0x8...,0.817147,0.0,524763,1638026591,2021-11-27 15:23:11,vETH,3990.155285,4288.026265,0.000000
2,0x98029c23e4a554989c280de7fcb1c2ededf2b17f-0x8...,9.275207,0.0,539423,1638065839,2021-11-28 02:17:19,vBTC,54255.244326,55240.631335,0.000000
3,0x98029c23e4a554989c280de7fcb1c2ededf2b17f-0x8...,0.272230,0.0,547656,1638084887,2021-11-28 07:34:47,vBTC,52968.683273,54581.737969,0.000000
4,0x98029c23e4a554989c280de7fcb1c2ededf2b17f-0x8...,0.322718,0.0,551150,1638093044,2021-11-28 09:50:44,vBTC,53287.434734,54581.737969,0.000000
...,...,...,...,...,...,...,...,...,...,...
16209,0x4b5aeeb5fd6b9d7febf8aec97bf5717423f8cc89-0x8...,0.000000,0.0,11553713,1655117474,2022-06-13 10:51:14,vETH,980.100995,1223.712468,1635.414776
16210,0xd3fab52a0c93346f23a9b9b24fd45ea0b8dd3420-0x3...,144.993911,0.0,11554393,1655117811,2022-06-13 10:56:51,vAAVE,59.489255,71.649517,0.000000
16211,0x377e6326a8e2ce4a856d31605ca33505ad2b57b1-0x9...,0.000000,0.0,11554604,1655117891,2022-06-13 10:58:11,vAPE,0.347863,34.876820,3917.139686
16212,0xd3fab52a0c93346f23a9b9b24fd45ea0b8dd3420-0x3...,0.000000,0.0,11554671,1655117952,2022-06-13 10:59:12,vAAVE,58.779686,70.371427,152196.723027


# Making a master table

In [5]:
master_maker_df = liquidity_changes_df

master_maker_df = pd.concat([master_maker_df, open_orders_df])
master_maker_df.sort_values("block_number")
master_maker_df = master_maker_df.reset_index()
master_maker_df.drop(columns=["index"], inplace=True)

master_maker_df

,id,maker,base,quote,quote_fee,block_number_log_index,timestamp,time,day_of_year,pool_name,lower_bound,upper_bound,fixed_liquidity,central_bound,collected_fee,collected_fee_in_lifecycle,block_number
0,0xec1e410b931f920c4aa3ae105e63463142b310fbe959...,0x9c1e0c67aa30c063f341885b12cb81cc94613fc7,0.000921,50.000000,0.0,515552004.0,1638000985,2021-11-27 08:16:25,331.0,vBTC,2.954278e-39,3.384921e+38,0.214579,1.692461e+38,NaN,NaN,NaN
1,0x7dba9c535f6c595ab39eeec818d89b34783442bafa81...,0x9c1e0c67aa30c063f341885b12cb81cc94613fc7,0.012181,50.000000,0.0,515554004.0,1638000985,2021-11-27 08:16:25,331.0,vETH,2.954278e-39,3.384921e+38,0.780407,1.692461e+38,NaN,NaN,NaN
2,0xc438a7cd8fee202ce2b53761811ef05f85095933b27e...,0x9c1e0c67aa30c063f341885b12cb81cc94613fc7,3.572179,200000.000000,0.0,515555003.0,1638000985,2021-11-27 08:16:25,331.0,vBTC,5.171263e+04,5.692287e+04,35648.835497,5.431775e+04,NaN,NaN,NaN
3,0x7871b4fe6113ac31c8be911146b109575cfb9ee90b13...,0x9c1e0c67aa30c063f341885b12cb81cc94613fc7,46.254982,200000.000000,0.0,515557003.0,1638000985,2021-11-27 08:16:25,331.0,vETH,3.895536e+03,4.313830e+03,120851.329740,4.104683e+03,NaN,NaN,NaN
4,0x653f9413a9c8df27d2800bbb71f06fd20b6d825b3d38...,0x35952141922ee99a04e016c967c3f0ad589ea958,0.100000,472.798469,0.0,516478004.0,1638003992,2021-11-27 09:06:32,331.0,vETH,3.990155e+03,4.288026e+03,386.315245,4.139091e+03,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
146268,0x4b5aeeb5fd6b9d7febf8aec97bf5717423f8cc89-0x8...,NaN,NaN,NaN,NaN,NaN,1655117474,2022-06-13 10:51:14,NaN,vETH,9.801010e+02,1.223712e+03,1635.414776,NaN,0.000000,0.0,11553713.0
146269,0xd3fab52a0c93346f23a9b9b24fd45ea0b8dd3420-0x3...,NaN,NaN,NaN,NaN,NaN,1655117811,2022-06-13 10:56:51,NaN,vAAVE,5.948926e+01,7.164952e+01,0.000000,NaN,144.993911,0.0,11554393.0
146270,0x377e6326a8e2ce4a856d31605ca33505ad2b57b1-0x9...,NaN,NaN,NaN,NaN,NaN,1655117891,2022-06-13 10:58:11,NaN,vAPE,3.478628e-01,3.487682e+01,3917.139686,NaN,0.000000,0.0,11554604.0
146271,0xd3fab52a0c93346f23a9b9b24fd45ea0b8dd3420-0x3...,NaN,NaN,NaN,NaN,NaN,1655117952,2022-06-13 10:59:12,NaN,vAAVE,5.877969e+01,7.037143e+01,152196.723027,NaN,0.000000,0.0,11554671.0
